<a href="https://colab.research.google.com/github/holykov0816/public/blob/main/Voting_Bot_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
import random
import telebot
from telebot import types

# Replace 'YOUR_BOT_TOKEN' with your actual Telegram bot token
bot_token = '7204606124:AAElm50pGDfIOrB8_Dne_InZNFIahaAwVeg'
bot = telebot.TeleBot(bot_token)

# Dictionary to store uploaded photos and votes
photos = {}
user_votes = {}

# Constants
PHOTOS_PER_PAGE = 5

# Assign each photo a unique index to use in callback data
photo_refs = {}

# Start command handler
@bot.message_handler(commands=['start'])
def start(message):
    # Setting up a compact and visually appealing button layout
    markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    upload_button = types.KeyboardButton('🖼️ Upload Photo')
    vote_button = types.KeyboardButton('🗳️ Vote for Outfits')
    markup.add(upload_button, vote_button)

    # Enhanced greeting message
    greeting_text = (
        "👻 *Welcome to the Vampire Costume Contest Bot!* 🎃\n\n"
        "Here is how to join to the fun:\n\n"
        "🧛‍♀️ *Upload a photo* of your Halloween costume with '🖼️ Upload Photo'.\n"
        "💀 *Vote* for the scariest outfits with '🗳️ Vote for Outfits'.\n\n"
        "The top 3 outfits will advance to the final round, and a winner will be chosen by the crowd!\n\n"
        "1 place - 30 EUR\n"
        "2 place - 15 EUR\n"
        "3 place - 5 EUR\n\n"
        "📊 Use /all\\_votes to check current rankings and see who’s winning.\n\n"
        "Let's find out who's the *bloodiest vampire*! 🩸🧛"
    )

    bot.send_message(
        message.chat.id,
        greeting_text,
        parse_mode="Markdown",
        reply_markup=markup
    )


@bot.message_handler(commands=['top_photo'])
def top_photo(message):
    if not photos:
        bot.send_message(message.chat.id, "No photos have been submitted yet.")
        return

    sorted_photos = sorted(photos.items(), key=lambda item: item[1]['votes'], reverse=True)
    top_photos = sorted_photos[:3]

    if all(photo[1]['votes'] == 0 for photo in top_photos):
        bot.send_message(message.chat.id, "No votes have been cast yet.")
        return

    for i, (photo_id, info) in enumerate(top_photos, start=1):
        vote_count = info['votes']
        caption = f"🏆 Rank {i}: {vote_count} votes"
        try:
            bot.send_photo(message.chat.id, photo=photo_id, caption=caption)
            print(f"Displayed photo with ID: {photo_id} - Rank {i} with {vote_count} votes.")
        except Exception as e:
            print(f"Error sending photo with ID {photo_id}: {e}")

@bot.message_handler(commands=['set_menu'])
def set_menu(message):
    commands = [
        types.BotCommand("start", "Show introduction and instructions"),
        types.BotCommand("all_votes", "View current rankings"),
        types.BotCommand("top_photo", "View top 3 photos"),
    ]
    bot.set_my_commands(commands)
    bot.send_message(message.chat.id, "Menu commands set! Use the quick access menu for functions.")

@bot.message_handler(commands=['all_votes'])
def all_votes(message):
    if not photos:
        bot.send_message(message.chat.id, "No photos have been submitted yet.")
        return

    sorted_photos = sorted(photos.items(), key=lambda item: item[1]['votes'], reverse=True)
    for rank, (photo_id, info) in enumerate(sorted_photos, start=1):
        vote_count = info['votes']
        caption = f"📊 Rank {rank}: {vote_count} votes"
        try:
            bot.send_photo(message.chat.id, photo=photo_id, caption=caption)
            print(f"Displayed photo with ID: {photo_id} - Rank {rank} with {vote_count} votes.")
        except Exception as e:
            print(f"Error sending photo with ID {photo_id}: {e}")

@bot.message_handler(content_types=['text'])
def handle_text(message):
    if message.text == '🖼️ Upload Photo':
        bot.send_message(message.chat.id, "Please upload a photo of your costume!")
    elif message.text == '🗳️ Vote for Outfits':
        send_photo_page(message.chat.id, 0)

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    photo_id = message.photo[-1].file_id
    short_ref = str(len(photo_refs))
    photo_refs[short_ref] = photo_id
    photos[photo_id] = {'user_id': message.from_user.id, 'votes': 0}

    bot.send_message(message.chat.id, "Your photo has been submitted for the contest!")
    print(f"Photo uploaded with short ref: {short_ref} and ID: {photo_id}")

def send_photo_page(chat_id, page):
    photo_ids = list(photo_refs.keys())
    random.shuffle(photo_ids)
    start = page * PHOTOS_PER_PAGE
    end = start + PHOTOS_PER_PAGE
    selected_photos = photo_ids[start:end]

    if not selected_photos:
        bot.send_message(chat_id, "No photos have been submitted yet.")
        return

    for short_ref in selected_photos:
        photo_id = photo_refs[short_ref]
        markup = types.InlineKeyboardMarkup()
        vote_button = types.InlineKeyboardButton("🗳️ Vote", callback_data=short_ref)
        markup.add(vote_button)

        try:
            bot.send_photo(chat_id, photo=photo_id, caption="Vote for this outfit!", reply_markup=markup)
            print(f"Sending photo with short ref: {short_ref} for voting.")
        except Exception as e:
            print(f"Error sending photo ID: {photo_id}, error: {e}")

    pagination_markup = types.InlineKeyboardMarkup()
    if page > 0:
        prev_button = types.InlineKeyboardButton("⬅️ Previous", callback_data=f"prev_{page - 1}")
        pagination_markup.add(prev_button)
    if end < len(photo_ids):
        next_button = types.InlineKeyboardButton("Next ➡️", callback_data=f"next_{page + 1}")
        pagination_markup.add(next_button)

    bot.send_message(chat_id, "Use the buttons below to navigate photos.", reply_markup=pagination_markup)

@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    if call.data in photo_refs:
        photo_id = photo_refs[call.data]
        uploader_id = photos[photo_id]['user_id']

        if call.from_user.id == uploader_id:
            bot.answer_callback_query(call.id, "You cannot vote for your own photo!")
            return

        if call.from_user.id not in user_votes:
            user_votes[call.from_user.id] = set()

        if photo_id not in user_votes[call.from_user.id]:
            user_votes[call.from_user.id].add(photo_id)
            photos[photo_id]['votes'] += 1
            bot.answer_callback_query(call.id, "Thanks for voting!")
            print(f"User {call.from_user.id} voted for photo {photo_id}. Total votes: {photos[photo_id]['votes']}")
        else:
            bot.answer_callback_query(call.id, "You have already voted for this photo.")

    elif call.data.startswith("prev_") or call.data.startswith("next_"):
        page = int(call.data.split("_")[1])
        bot.delete_message(call.message.chat.id, call.message.message_id)
        send_photo_page(call.message.chat.id, page)

# Polling
bot.polling(none_stop=True)
